In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql import Row

In [2]:
mylist = [
  {"id":1,"name":"sally", "role": "manager", "updated_at": "2020-01-13T18:42:21Z"},
  {"id":1,"name":"sally", "role": "director", "updated_at": "2020-01-14T10:17:45Z"},
  {"id":2,"name":"eric", "role": "leader", "updated_at": "2020-01-28T07:33:58Z"},
  {"id":3,"name":"john", "role": "leader", "updated_at": "2020-01-01T20:00:00Z"},
  {"id":2,"name":"eric", "role": "developer", "updated_at": "2020-01-10T08:28:31Z"},
]

df = spark.createDataFrame(Row(**x) for x in mylist)


df.show()

+---+-----+---------+--------------------+
 id| name| role| updated_at|
+---+-----+---------+--------------------+
 1|sally| manager|2020-01-13T18:42:21Z|
 1|sally| director|2020-01-14T10:17:45Z|
 2| eric| leader|2020-01-28T07:33:58Z|
 3| john| leader|2020-01-01T20:00:00Z|
 2| eric|developer|2020-01-10T08:28:31Z|
+---+-----+---------+--------------------+

In [3]:
def metadata(name): #function for getting metadata in a dict
  null=[str(n.nullable) for n in name.schema.fields] #nullability
  types=[str(i.dataType) for i in name.schema.fields] #type 
  both = [list(a)[] for a in zip(types,null)]
  #combine type+nullability
  names= name.columns #names of columns
  final = {} #create dict
  for key in names: 
     for value in both: 
          final[key] = value
          both.remove(value)
          break
  return final


metadata(df)

Out[11]: {'id': ['LongType', 'True'],
 'name': ['StringType', 'True'],
 'role': ['StringType', 'True'],
 'updated_at': ['StringType', 'True']}

In [4]:
w=Window().partitionBy("id").orderBy(F.col("updated_at").desc())
df1=df.withColumn("updated_at",F.to_timestamp("updated_at")).withColumn("max", F.row_number().over(w)).filter(F.col("max")==1).drop("max").orderBy("id")
df1.toJSON().collect()


Out[16]: ['{"id":1,"name":"sally","role":"director","updated_at":"2020-01-14T10:17:45.000Z"}',
 '{"id":2,"name":"eric","role":"leader","updated_at":"2020-01-28T07:33:58.000Z"}',
 '{"id":3,"name":"john","role":"leader","updated_at":"2020-01-01T20:00:00.000Z"}']